In [46]:
import requests
import pandas as pd
import tensorflow as tf
import numpy as np
import base64
import json


import data

In [47]:
data = pd.read_csv("../Submmison_Akhir/Obesity Classification.csv")


drop column ID dan save dataset ke folder data

In [48]:

# Drop kolom 'ID'
data = data.drop(columns=['ID'])

# Simpan ke file CSV baru
data.to_csv("data/data.csv", index=False)

In [49]:
data

,Age,Gender,Height,Weight,BMI,Label
0,25,Male,175,80,25.3,Normal Weight
1,30,Female,160,60,22.5,Normal Weight
2,35,Male,180,90,27.3,Overweight
3,40,Female,150,50,20.0,Underweight
4,45,Male,190,100,31.2,Obese
...,...,...,...,...,...,...
103,11,Male,175,10,3.9,Underweight
104,16,Female,160,10,3.9,Underweight
105,21,Male,180,15,5.6,Underweight
106,26,Female,150,15,5.6,Underweight


melihat berapa jumlah label

In [50]:
data["Label"].unique()

array(['Normal Weight', 'Overweight', 'Underweight', 'Obese'],
      dtype=object)

melihat metrix evaluasi setelah pipeline berhasil dijalankan

In [51]:
from pathlib import Path
import tensorflow_model_analysis as tfma
import pandas as pd

# === 1️⃣ Path evaluator ===
eval_result_path = Path('../Submmison_Akhir/output/obesity-pipeline/Evaluator/evaluation/8')
print("Folder path:", eval_result_path)

# === 2️⃣ Cek isi folder ===
if eval_result_path.exists():
    files = [f.name for f in eval_result_path.iterdir()]
    print("Isi folder:", files)
else:
    print("⚠️ Folder tidak ditemukan! Pastikan path sudah benar.")

# === 3️⃣ Load eval result ===
eval_result = tfma.load_eval_result(str(eval_result_path))
print("✅ Eval result berhasil dimuat.\n")

# === 4️⃣ Tampilkan slice dan detail isi metrics[''] ===
for (slice_key, metrics) in eval_result.slicing_metrics:
    # Nama slice
    if slice_key == ():
        slice_name = "Overall"
    else:
        slice_name = ", ".join([f"{k}={v}" for k, v in slice_key])
    
    print(f"Slice: {slice_name}")
    print("Metrics keys:", list(metrics.keys()))

    # Kalau ada key kosong '', print isinya
    if '' in metrics:
        print("Isi metrics['']:")
        print(metrics[''])
    print("="*70)


Folder path: ..\Submmison_Akhir\output\obesity-pipeline\Evaluator\evaluation\8
Isi folder: ['attributions-00000-of-00001.tfrecord', 'eval_config.json', 'metrics-00000-of-00001.tfrecord', 'plots-00000-of-00001.tfrecord', 'validations.tfrecord']
✅ Eval result berhasil dimuat.

Slice: Gender_xf=0.0
Metrics keys: ['']
Isi metrics['']:
{'': {'sparse_categorical_accuracy': {'doubleValue': 0.9523809552192688}, 'loss': {'doubleValue': 0.7675824165344238}, 'example_count': {'doubleValue': 21.0}, 'auc': {'doubleValue': 0.9674981103552531}, 'precision': {'doubleValue': 0.9523809523809523}, 'recall': {'doubleValue': 0.9523809523809523}}}
Slice: Overall
Metrics keys: ['']
Isi metrics['']:
{'': {'sparse_categorical_accuracy': {'doubleValue': 0.9523809552192688}, 'loss': {'doubleValue': 0.7675824165344238}, 'example_count': {'doubleValue': 21.0}, 'auc': {'doubleValue': 0.9674981103552531}, 'precision': {'doubleValue': 0.9523809523809523}, 'recall': {'doubleValue': 0.9523809523809523}}}
Slice: Gender_

testing deployment url

In [52]:
print(requests.get("https://obesityclassification-production.up.railway.app/v1/models/cc-model").json())


{'model_version_status': [{'version': '1751809833', 'state': 'AVAILABLE', 'status': {'error_code': 'OK', 'error_message': ''}}]}


In [53]:

def prepare_json(inputs: dict):

    # Mendefinisikan mapping dari kunci input ke jenis Fitur TensorFlow
    feature_mapping = {
    "Age": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['Age'])])),
    "Gender": tf.train.Feature(bytes_list=tf.train.BytesList(value=[inputs['Gender'].encode()])),
    "BMI": tf.train.Feature(float_list=tf.train.FloatList(value=[float(inputs['BMI'])])),
    "Height": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['Height'])])),
    "Weight": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['Weight'])])),
    }


    # Membuat TensorFlow Example
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_mapping)
    ).SerializeToString()

    # Melakukan encode Example ke base64
    result = {
        "examples": {
            "b64": base64.b64encode(example).decode()
        }
    }

    # Mengembalikan data dalam format JSON
    return json.dumps({
        "signature_name": "serving_default",
        "instances": [result],
    })


# Fungsi untuk memprediksi data
LABELS = ['Normal Weight', 'Overweight', 'Underweight', 'Obese']

def predict(inputs):
    json_data = prepare_json(inputs)
    endpoint = "https://obesityclassification-production.up.railway.app/v1/models/cc-model:predict"

    try:
        response = requests.post(endpoint, data=json_data, timeout=10) 
        response.raise_for_status() 
        res_json = response.json()
        print("Response JSON:", res_json)

        # Ambil array prediksi
        predictions = res_json.get("predictions", [])
        if not predictions or not isinstance(predictions[0], list):
            print("Format prediksi tidak sesuai.")
            return None

        probs = predictions[0]
        predicted_index = int(np.argmax(probs))
        predicted_label = LABELS[predicted_index]

        print(f"Predicted label: {predicted_label}, Probabilities: {probs}")

        return predicted_label

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - Response content: {response.text}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An unexpected error occurred: {req_err}")

    return None




In [54]:
datas = data.drop(columns=["Label"], axis=1)

Prediksi data berdasarkan baris ke 105 dan 1

In [55]:
oke = predict(datas.iloc[105])
oke

Response JSON: {'predictions': [[1.0, 0.0, 0.0, 0.0]]}
Predicted label: Normal Weight, Probabilities: [1.0, 0.0, 0.0, 0.0]


'Normal Weight'

In [56]:
oke = predict(datas.iloc[1])
oke

Response JSON: {'predictions': [[6.20193605e-19, 1.0, 1.11582858e-21, 0.0]]}
Predicted label: Overweight, Probabilities: [6.20193605e-19, 1.0, 1.11582858e-21, 0.0]


'Overweight'